In [32]:
import pandas as pd
import sklearn as sk
import numpy as np

In [5]:
data = pd.read_csv("bank_customer_churn.csv")
data = data.drop(["RowNumber"],axis=1)

In [6]:
data.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [8]:
# See if I have to dummize the surname column
# Seems no.
data["Surname"].value_counts()

Smith          32
Scott          29
Martin         29
Walker         28
Brown          26
Shih           25
Genovese       25
Yeh            25
Maclean        24
Wright         24
White          23
Ma             23
Wilson         23
Fanucci        23
Moore          22
Chu            22
Wang           22
Lu             22
Johnson        22
McGregor       21
Sun            21
Thompson       21
Mai            21
Miller         20
Hughes         20
Lo             20
Trevisani      20
Jamieson       20
Graham         20
Kennedy        20
               ..
Sanchez         1
Peavy           1
Souter          1
Outhwaite       1
Humffray        1
Potts           1
Bales           1
Donoghue        1
Armfield        1
Abrego          1
Higinbotham     1
Kendall         1
Ardis           1
Steere          1
Ingle           1
Eidson          1
Guerin          1
Brierly         1
Mitchel         1
Schatz          1
Lea             1
Frederick       1
Morant          1
Nyhan           1
Nicolay   

In [9]:
# See if I have to dummize the surname column
# Seems yes.
data["Geography"].value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

In [10]:
todummy = ['Geography','Gender']
# Get Dummies
data = pd.get_dummies(data, columns=todummy)

In [11]:
# Check if that's going well
data.head()

,CustomerId,Surname,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,15634602,Hargrave,619,42,2,0.00,1,1,1,101348.88,1,1,0,0,1,0
1,15647311,Hill,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1,1,0
2,15619304,Onio,502,42,8,159660.80,3,1,0,113931.57,1,1,0,0,1,0
3,15701354,Boni,699,39,1,0.00,2,0,0,93826.63,0,1,0,0,1,0
4,15737888,Mitchell,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1,1,0


In [22]:
# Do the split on columns
X = data.drop(['CustomerId','Surname','Exited'],axis=1)
Y = data['Exited']

In [27]:
# Do the random split
from sklearn.cross_validation import train_test_split
X_train,X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.5)

In [29]:
# Scales the data
# Source: http://scikit-learn.org/stable/modules/neural_networks_supervised.html
# Section 1.17.8
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [47]:
# Using NN
# First trying out the official example
# Source: http://scikit-learn.org/stable/modules/neural_networks_supervised.html
# Section 1.17.2
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(X_train, Y_train)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [31]:
clf.predict(X_test)

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [34]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection  import cross_val_score
cross_fold_accr = cross_val_score(clf, X, Y, scoring="accuracy", cv=5)
print(np.mean(cross_fold_accr))

0.7963000592500148


In [51]:
# Using Grid Search To Find the Best Parameter automatically
from sklearn.grid_search import GridSearchCV
a = 10.0 ** -np.arange(1, 7)
param_grids = {'activation':['identity', 'logistic', 'tanh', 'relu'],'alpha':a, 'hidden_layer_sizes':(1, 2, 3,4,5)}
grid = GridSearchCV(MLPClassifier(), param_grid=param_grids, cv=5)
grid.fit(X,Y)
grid.grid_scores_, grid.best_params_, grid.best_score_

([mean: 0.61460, std: 0.09212, params: {'activation': 'identity', 'alpha': 0.1, 'hidden_layer_sizes': 1},
  mean: 0.69200, std: 0.07198, params: {'activation': 'identity', 'alpha': 0.1, 'hidden_layer_sizes': 2},
  mean: 0.70150, std: 0.07101, params: {'activation': 'identity', 'alpha': 0.1, 'hidden_layer_sizes': 3},
  mean: 0.69400, std: 0.10572, params: {'activation': 'identity', 'alpha': 0.1, 'hidden_layer_sizes': 4},
  mean: 0.67980, std: 0.09672, params: {'activation': 'identity', 'alpha': 0.1, 'hidden_layer_sizes': 5},
  mean: 0.63520, std: 0.20870, params: {'activation': 'identity', 'alpha': 0.01, 'hidden_layer_sizes': 1},
  mean: 0.60340, std: 0.17329, params: {'activation': 'identity', 'alpha': 0.01, 'hidden_layer_sizes': 2},
  mean: 0.75210, std: 0.04009, params: {'activation': 'identity', 'alpha': 0.01, 'hidden_layer_sizes': 3},
  mean: 0.68460, std: 0.15778, params: {'activation': 'identity', 'alpha': 0.01, 'hidden_layer_sizes': 4},
  mean: 0.69330, std: 0.07994, params: {'a

In [63]:
# Using the best parameter to create ROC curve

best_clf = MLPClassifier(activation='tanh', alpha=0.0001, hidden_layer_sizes=3)
best_clf.fit(X_train, Y_train)

from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

Y_test_probabilities = best_clf.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(Y_test, Y_test_probabilities)
auc = roc_auc_score(Y_test, Y_test_probabilities)
print(auc)

0.5102742496561767


In [58]:
# Compare to a regular decision tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
tree = DecisionTreeClassifier(criterion="entropy")
tree.fit(X_train, Y_train)
Y_predicted = tree.predict(X_test)
accuracy = accuracy_score(Y_predicted, Y_test)
print('Accuracy =',accuracy)

Y_test_probabilities = tree.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(Y_test, Y_test_probabilities)
auc = roc_auc_score(Y_test, Y_test_probabilities)
print('AUC =',auc)

Accuracy = 0.6706
AUC = 0.5048323066788402


## Conclusion
Compared to Decision Tree, Multi-layer perceptron NN (MLP) with Stochastic Gradient Descent can get a better result in accuracy, but a lower AUC for strange reasons.